# Pythonic SysML Elements

This notebook walks through the approach to making SysML elements act Pythonically while in the PyMBE environment in order to support more intuitive codes that inspect and analyze SysML models.

Note that the Markdown headers for code snippets are set up to be translated into docstrings for documentation of the code eventually.

In [1]:
import inspect
import os
import traceback
import logging
import pymbe.api as pm
from pymbe.model import Element
from pymbe.model import ListOfNamedItems
from pymbe.model import Model

import json
from dataclasses import dataclass, field
from typing import Any, Dict, List, Set, Tuple, Union, Collection
from collections import defaultdict

from uuid import uuid4

with open('example_data/sysml_3a.json', 'r') as fp:
    element_raw_data = json.load(fp)
    
with open('canonical_data/Model_Loader_Test_Level_1.json', 'r') as fp:
    element_raw_post_data = json.load(fp)


In [2]:
"ownedMember"[5:]

'Member'

In [3]:
logging.basicConfig(level=logging.INFO)

## Lifecycle Tracker

The element class below is created to track the full lifecycle of an Element as it is read back from Python, populated with attribute data, and related to others through serialized relations references.

In [4]:
class InstrumentedElement(Element):
    
    def __post_init__(self):
        logging.info("[Element] entering post initialization.")
        
        #super().__post_init__()
        if not self._model._initializing:
            self._model.elements[self._id] = self
        if self._data:
            logging.info("[Element] data found. About to resolve.")
            self.resolve()
            
    def __setattr__(self, name, value):
        super().__setattr__(name, value)
        logging.info(f"[Element] value of {name} is trying to be set to {value}")
            
    def resolve(self):
        if not self._is_proxy:
            logging.info("[Element] element is not proxy.")
            return
        
        logging.info("[Element] element is still in proxy mode - resolving internal data")

        model = self._model
        if not self._data:
            logging.info("[Element] no data found to resolve.")
            if not model._api:
                raise SystemError("Model must have an API to retrieve the data from!")
            self._data = model._api.get_element_data(self._id)
        data = self._data
        self._id = data["@id"]
        self._metatype = data["@type"]

        self._is_abstract = bool(data.get("isAbstract"))
        self._is_relationship = bool(data.get("relatedElement"))
        
        logging.info(f"[Element] assigned id = {self._id}, metatype = {self._metatype}, is abstract = {self._is_abstract}, and is relationship = {self._is_relationship}.")
        
        for key, items in data.items():
            if key.startswith("owned") and isinstance(items, list):
                logging.info(f"Key starting with owned found: {key}")
                data[key] = ListOfNamedItems(items)
                logging.info(f"Data resolved to: {data[key]}")
        if not model._initializing:
            self._model._add_element(self)
        self._is_proxy = False

## Test Objects

Load up an instrumented element and model to explore the concepts below.

In [5]:
def factor_element_post_data_to_get(post_element):
    return {**{k: v for k, v in post_element["payload"].items()}, "@id": post_element["identity"]["@id"]}

In [6]:
factor_element_post_data_to_get(element_raw_post_data[0])

{'elementId': 'f48aa78b-6c4e-4826-a4a7-9c77cde2b9d0',
 'annotation': [],
 'owningRelationship': {'@id': '91d3a7d2-0834-4aa0-9a79-dc2a21fb11b9'},
 'aliasIds': [],
 '@type': 'Documentation',
 'ownedRelationship': [],
 'body': 'Meant to cover simple loading of classifiers, features, FeatureOwnership, MemberOwnership, Package, FeatureTyping, Specialization.',
 'isImpliedIncluded': False,
 '@id': 'f48aa78b-6c4e-4826-a4a7-9c77cde2b9d0'}

In [7]:
factored_post_data = [factor_element_post_data_to_get(raw_post) for raw_post in element_raw_post_data]

In [8]:
def get_index_of_named_element(name="example"):
    key_element_index = 0
    max_tries = 1000
    found = False
    while not found and key_element_index < max_tries:
        if "declaredName" in element_raw_post_data[key_element_index]["payload"]:
            if element_raw_post_data[key_element_index]["payload"]["declaredName"] == name:
                found = True
                return key_element_index
        key_element_index = key_element_index + 1

In [9]:
get_index_of_named_element(name="Bare Classed Feature")

15

In [10]:
element_raw_post_data[15]

{'payload': {'elementId': '231cb543-a22d-4347-82c2-3bb910a7a810',
  'isDerived': False,
  'owningRelationship': {'@id': '1e219b62-6777-4d0d-93d0-09afaaf97570'},
  'aliasIds': [],
  '@type': 'Feature',
  'ownedRelationship': [{'@id': '024d0e2d-dee8-47ed-a9ba-0bacbc38a201'}],
  'isUnique': True,
  'isPortion': False,
  'isAbstract': False,
  'isEnd': False,
  'isImpliedIncluded': False,
  'isComposite': False,
  'isReadOnly': False,
  'declaredName': 'Bare Classed Feature',
  'isSufficient': False,
  'isOrdered': False},
 'identity': {'@id': '231cb543-a22d-4347-82c2-3bb910a7a810'}}

In [11]:
@dataclass(repr=False)
class InstrumentedModel(Model):
    
    instrumented_name: str = "Bare Classed Feature"
    instrumented_element: InstrumentedElement = None
    
    def __post_init__(self):
        '''Same as other code but with insertion of instrumented element'''
        
        self._load_metahints()
        
        logging.info(f"[Model] loading model with instrumented element with name '{self.instrumented_name}'")
        
        instrumented_data = None
        instrumented_element = None
        for id_, data in self.elements.items():
            if "declaredName" in data and data["declaredName"] == self.instrumented_name:
                instrumented_data = (id_, data)
                
        logging.info(f"[Model] Model data is '{instrumented_data[1]}'")
                
        self.elements = {
            id_: Element(_data=data,
                         _model=self,
                         _metamodel_hints={att[0]: att[1:] for att in self._metamodel_hints[data["@type"]]})
            for id_, data in self.elements.items()
            if isinstance(data, dict)
        }
        
        self.instrumented_element = InstrumentedElement(_data=instrumented_data[1],
                                                        _model=self,
                                                        _metamodel_hints={att[0]: att[1:] for att in self._metamodel_hints[instrumented_data[1]["@type"]]})
        
        self.elements.update({instrumented_data[0]: self.instrumented_element})

        self._add_owned()

        # Modify and add derived data to the elements
        self._add_relationships()
        self._add_labels()
        self._initializing = False
        
    @staticmethod
    def load(
        elements: Collection[Dict],
        **kwargs,
    ) -> "Model":
        """Make a Model from an iterable container of elements"""
        return InstrumentedModel(
            elements={element["@id"]: element for element in elements},
            **kwargs,
        )
    
    def _add_relationships(self):
        """Adds relationships to elements"""
        relationship_mapper = {
            "through": ("source", "target"),
            "reverse": ("target", "source"),
        }
        # TODO: make this more elegant...  maybe.
        for relationship in self.all_relationships.values():
            endpoints = {
                endpoint_type: [
                    self.get_element(endpoint["@id"])
                    for endpoint in relationship._data[endpoint_type]
                ]
                for endpoint_type in ("source", "target")
            }
            metatype = relationship._metatype
            for direction, (key1, key2) in relationship_mapper.items():
                endpts1, endpts2 = endpoints[key1], endpoints[key2]
                for endpt1 in endpts1:
                    for endpt2 in endpts2:
                        endpt1._derived[f"{direction}{metatype}"] += [{"@id": endpt2._data["@id"]}]
                        if endpt1 == self.instrumented_element:
                            logging.info(f"[Model] updating derived field of instrument element for {direction}{metatype}")

In [12]:
test_model = InstrumentedModel.load(elements=factored_post_data)

INFO:root:[Model] loading model with instrumented element with name 'Bare Classed Feature'
INFO:root:[Model] Model data is '{'elementId': '231cb543-a22d-4347-82c2-3bb910a7a810', 'isDerived': False, 'owningRelationship': {'@id': '1e219b62-6777-4d0d-93d0-09afaaf97570'}, 'aliasIds': [], '@type': 'Feature', 'ownedRelationship': [{'@id': '024d0e2d-dee8-47ed-a9ba-0bacbc38a201'}], 'isUnique': True, 'isPortion': False, 'isAbstract': False, 'isEnd': False, 'isImpliedIncluded': False, 'isComposite': False, 'isReadOnly': False, 'declaredName': 'Bare Classed Feature', 'isSufficient': False, 'isOrdered': False, '@id': '231cb543-a22d-4347-82c2-3bb910a7a810'}'
INFO:root:[Element] value of _data is trying to be set to {'elementId': '231cb543-a22d-4347-82c2-3bb910a7a810', 'isDerived': False, 'owningRelationship': {'@id': '1e219b62-6777-4d0d-93d0-09afaaf97570'}, 'aliasIds': [], '@type': 'Feature', 'ownedRelationship': [{'@id': '024d0e2d-dee8-47ed-a9ba-0bacbc38a201'}], 'isUnique': True, 'isPortion': Fals

## Interchange data and loading

The API for standing up a model in memory is built around loading data from Python dictionaries. This is mostly based around major APIs of interest (e.g., the SysML v2 REST API) exchanging data in the JSON format with a main set of keys and values.

### Bulk loading model data

The entry point to loading up the models is a bulk load method that gathers collections of dictionaries (each model element is expected to serialize as a dictionary from JSON) and creates a new dictionary that allows for rapid look up of elements by their IDs.

### ID reference format

The serialized JSON files will present IDs as a single entry dictionary of the form {'@id': uuid}.

In [13]:
pm.is_id_item({"@id": "2012_ajfal_jfajkl"})

True

## Pythonic Model elements

The methods and the class developed here are intended to support the Python representation and manipulation of system model data. The partial methods describe below set up various fields in objects of the Element class for use in downstream or user codes involving this library.

There are a variety of features developed here to make the interaction with system model data more Pythonic.

### List of Items Accessible by Name

For convenience, a class is introduced that allows for collections of objects to be referenced by name rather than by unique identifier. This requires that names in a given list are unique.

The ListOfNamedItemsClass extends list to make names within model element data a useable index.

In [14]:
print(inspect.getsource(ListOfNamedItems.__getitem__))

    def __getitem__(self, key):
        item_map = {
            item._data["declaredName"]: item
            for item in self
            if isinstance(item, Element) and "declaredName" in item._data
        }
        effective_item_map = {
            item._data["effectiveName"]: item
            for item in self
            if isinstance(item, Element) and "effectiveName" in item._data
        }
        if key in item_map:
            return item_map[key]
        if key in effective_item_map:
            return effective_item_map[key]
        if isinstance(key, int):
            return super().__getitem__(key)
        return None



### Navigating fields for owned elements (and more specialized collections)

Elements should be able to directly reference elements through ownership relations. This is done by dynamically constructing Python fields for meta-attributes that start with "owned." This is done in the element read lifecycle, starting with when the model begins to load.

In [15]:
test_model.ownedElement

[<cdd5d1e3-fe4b-52bd-8a01-51a53f22ba47 «LibraryPackage»>,
 <eff08982-c95f-439f-8ee4-563136c74ed8 «Type»>,
 <3093cc44-3025-450e-8ea8-18705529b528 «Namespace»>]

In [16]:
test_model.ownedRelationship

[]

In [17]:
test_model.elements["231cb543-a22d-4347-82c2-3bb910a7a810"]._derived

defaultdict(list,
            {'throughFeatureTyping': [{'@id': '2d68b91c-5f85-4853-be98-d22ad3ea767c'}],
             'reverseOwningMembership': [{'@id': 'f6610ab3-807b-4335-a7bf-eb72e63df831'}],
             'label': 'Bare Classed Feature'})

In [18]:
test_model.instrumented_element

<231cb543-a22d-4347-82c2-3bb910a7a810 «Feature»>

In [19]:
test_model.instrumented_element._metamodel_hints

{'documentation': ['inherited', 'derived', 'Documentation', 'EReference'],
 'owningMembership': ['inherited',
  'derived',
  'OwningMembership',
  'EReference'],
 'ownedIntersecting': ['inherited', 'derived', 'Intersecting', 'EReference'],
 'chainingFeature': ['local', 'derived', 'Feature', 'EReference'],
 'ownedDisjoining': ['inherited', 'derived', 'Disjoining', 'EReference'],
 'ownedDifferencing': ['inherited', 'derived', 'Differencing', 'EReference'],
 'owningType': ['local', 'derived', 'Type', 'EReference'],
 'ownedRedefinition': ['local', 'derived', 'Redefinition', 'EReference'],
 'ownedFeature': ['inherited', 'derived', 'Feature', 'EReference'],
 'membership': ['inherited', 'derived', 'Membership', 'EReference'],
 'importedMembership': ['inherited', 'derived', 'Membership', 'EReference'],
 'featuringType': ['local', 'derived', 'Type', 'EReference'],
 'textualRepresentation': ['inherited',
  'derived',
  'TextualRepresentation',
  'EReference'],
 'unioningType': ['inherited', 'der

In [20]:
test_model.instrumented_element.declaredName

'Bare Classed Feature'

In [21]:
test_model.instrumented_element.type[0].declaredName

'My Bare Class'

In [22]:
test_model.instrumented_element.ownedRelationship[0]._data

{'elementId': '024d0e2d-dee8-47ed-a9ba-0bacbc38a201',
 'isImplied': False,
 'aliasIds': [],
 'typedFeature': {'@id': '231cb543-a22d-4347-82c2-3bb910a7a810'},
 '@type': 'FeatureTyping',
 'ownedRelationship': [],
 'source': [{'@id': '231cb543-a22d-4347-82c2-3bb910a7a810'}],
 'ownedRelatedElement': [],
 'specific': {'@id': '231cb543-a22d-4347-82c2-3bb910a7a810'},
 'type': {'@id': '2d68b91c-5f85-4853-be98-d22ad3ea767c'},
 'isImpliedIncluded': False,
 'target': [{'@id': '2d68b91c-5f85-4853-be98-d22ad3ea767c'}],
 'owningRelatedElement': {'@id': '231cb543-a22d-4347-82c2-3bb910a7a810'},
 'general': {'@id': '2d68b91c-5f85-4853-be98-d22ad3ea767c'},
 '@id': '024d0e2d-dee8-47ed-a9ba-0bacbc38a201'}

### Accessing fields Pythonically

The next thing to add to the Element class are magics to match the meta-attributes of loaded elements to properties on Python objects. That is done by adding to the getattr and getitem magics.

This should focus on the metafields with expected literal values in the JSON.

In [23]:
print(''.join(inspect.getsourcelines(Element.resolve)[0][16:-3]))

        for key, items in data.items():
            # set up owned elements to be referencable by their name
            if key.startswith("owned") and isinstance(items, list):
                data[key] = ListOfNamedItems(items)
            # add Pythonic property to Element object based on metamodel for primary data values
            elif (
                key in self._metamodel_hints
                and self._metamodel_hints[key][1] == "primary"
                and self._metamodel_hints[key][3] != "EReference"
            ):
                setattr(self, key, items)



In [24]:
test_model.elements["231cb543-a22d-4347-82c2-3bb910a7a810"].ownedRelationship

[<FeatureTyping([<231cb543-a22d-4347-82c2-3bb910a7a810 «Feature»>] ←→ [<2d68b91c-5f85-4853-be98-d22ad3ea767c «Classifier»>])>]

### Navigating relationships Pythonically

The next Pythonic task for our model elements is the ability to traverse SysML v2 relationships Pythonically, either in the forward (through) or reverse direction with simple property accessors.

In [25]:
def element_resolve_to_determine_element_as_relationship(self):
    self._is_relationship = "relatedElement" in data

In [26]:
 def model_add_relationships(self):
        """Adds relationships to elements"""
        relationship_mapper = {
            "through": ("source", "target"),
            "reverse": ("target", "source"),
        }
        # TODO: make this more elegant...  maybe.
        for relationship in self.all_relationships.values():
            endpoints = {
                endpoint_type: [
                    self.elements[endpoint["@id"]]
                    for endpoint in relationship._data[endpoint_type]
                ]
                for endpoint_type in ("source", "target")
            }
            metatype = relationship._metatype
            for direction, (key1, key2) in relationship_mapper.items():
                endpts1, endpts2 = endpoints[key1], endpoints[key2]
                for endpt1 in endpts1:
                    for endpt2 in endpts2:
                        endpt1._derived[f"{direction}{metatype}"] += [{"@id": endpt2._data["@id"]}]

### Dealing with derived attributes

In SysML v2 models, there are a large number of derived attributes and collections that can gather the incoming and outgoing relationships to other elements. These are actually the standard means of navigation rather than the relationship-based fields generated specifically for pyMBE (also to make it easier to remember some of the names).

In [27]:
ns = [oe for oe in test_model.ownedElement if oe._metatype == "Namespace"]
ns[0]._data

{'elementId': '3093cc44-3025-450e-8ea8-18705529b528',
 'aliasIds': [],
 '@type': 'Namespace',
 'ownedRelationship': [{'@id': '91d3a7d2-0834-4aa0-9a79-dc2a21fb11b9'},
  {'@id': '9c5cc8db-6b9b-4822-9d7c-5cab44ddaeb5'}],
 'isImpliedIncluded': False,
 '@id': '3093cc44-3025-450e-8ea8-18705529b528'}

In [28]:
ns[0].ownedRelationship[1].ownedRelatedElement[0]

<f6610ab3-807b-4335-a7bf-eb72e63df831 «Package»>

In [29]:
ns[0].ownedMember

[<f48aa78b-6c4e-4826-a4a7-9c77cde2b9d0 «Documentation»>,
 <f6610ab3-807b-4335-a7bf-eb72e63df831 «Package»>]

### Creating model elements Pythonically

After setting up a model in memory Pythonically, the next steps will be to create model elements within the model processing session.